In [21]:
import json
import re

In [2]:
s3 = json.load(open("../s3.json"))

In [25]:
def tabs(n):
    return "  " * n

def filter_operations(shapes):
   return { k:v for k, v in shapes.items() if v["type"] == "operation"}

def input_member_for_operation(operation_id):
   return operation_id + "Request"

# Clean a doc string for OpenAI's OpenAPI format.
# Notably, remove HTML tags, newlines and tabs, and truncate to 200 chars since that's the limit.
# https://platform.openai.com/docs/plugins/getting-started/openapi-definition
def clean_docstring(string): 
   tags_regex = re.compile('<.*?>') 
   return re.sub(tags_regex, '', string).replace("\n", "").replace("\t", "")[:200]

def convert_member_type(member_type):
  if member_type in ["blob", "timestamp", "enum"]:
    return "string"
  elif member_type == "long":
    return "integer"
  elif member_type == "structure":
    return "object"
  elif member_type == "list":
    return "array"
  else:
    return member_type

def write_openapi_header(f):
    f.write("""openapi: 3.0.1
info:
  title: TEST AWS SDK for Natural Language
  description: TEST Use AWS with natural language
  version: 'v1'
servers:
  - url: https://gpt4-plugin-test-mataslauzadis.vercel.app/
""")

def write_paths(f, shapes):
    f.write("paths:\n")

    shapes = filter_operations(shapes)
    shapes = { k:v for k, v in shapes.items() if k == "com.amazonaws.s3#PutObject" }

    for shape in shapes:
      endpoint = shapes[shape]["input"]["target"]  # i.e com.amazonaws.s3#AbortMultipartUploadRequest
      endpoint = endpoint.replace(".", "#")  # i.e com#amazonaws#s3#AbortMultipartUploadRequest
      endpoint = "/" + "/".join(endpoint.split("#"))  # i.e /com/amazonaws/s3/AbortMultipartUploadRequest

      f.write(tabs(1) + endpoint + ":\n")

      # Write HTTP Method (GET, POST, DELETE, etc.)
      # TODO Manually setting everything to POST
      http_trait = shapes[shape]["traits"].get("smithy.api#http")
      f.write(tabs(2) + "POST:\n")

      # Write Operation ID
      operation_id = endpoint.split("/")[-1]  # i.e AbortMultipartUploadRequest
      f.write(tabs(3) + f"operationId: {operation_id}\n")

      # Write summary
      documentation = shapes[shape]["traits"].get("smithy.api#documentation")
      if documentation is not None:
        # pass
        f.write(tabs(3) + f"summary: {clean_docstring(documentation)}\n")

      # Write input
      input_target = shapes[shape]["input"]["target"]
      f.write(tabs(3) + "requestBody:\n")
      f.write(tabs(4) + "required: true\n")
      f.write(tabs(4) + "content:\n")
      f.write(tabs(5) + "application/json:\n")
      f.write(tabs(6) + "schema:\n")
      f.write(tabs(7) + f"$ref: {input_target}\n")

      # Write responses (only specify HTTP 200 OK response for now)
      f.write(tabs(3) + "responses:\n")
      f.write(tabs(4) + "\"200\":\n")
      f.write(tabs(5) + "description: \"OK\"\n")
      f.write(tabs(5) + "content:\n")
      f.write(tabs(6) + "text:\n")
      f.write(tabs(7) + "\"OK\"\n")

      # Operation done, write newline separator
      f.write("\n")
      break

def write_schemas(f, shapes):
    f.write("components:\n")
    f.write(tabs(1) + "schemas:\n")

    # shapes = filter_operations(shapes)
    shapes_filtered = { k:v for k, v in shapes.items() if k == "com.amazonaws.s3#PutObject" }

    for shape in shapes_filtered:
      # Write schema ID
      input_target = shapes[shape]["input"]["target"]
      print("Writing schema for " + input_target)
      f.write(tabs(2) + input_target + ":\n")
      f.write(tabs(3) + "type: object\n")
      f.write(tabs(3) + "properties:\n")

      # Write schema body
      for member in shapes[input_target]["members"]:
        print("Processing " + member)
        write_member(shapes[input_target]["members"][member], shapes)

def write_terminal_member(f, member, shapes, num_tabs=4):
  member_type = shapes[member["target"]]["type"]

  # Write property name
  f.write(tabs(num_tabs) + member["target"] + ":\n")

  # Write property type
  f.write(tabs(num_tabs+1) + f"type: {convert_member_type(member_type)}\n")

  # Finish writing enum
  if member_type == "enum":
    f.write(tabs(num_tabs+1) + "enum: [")
    enum_shape = shapes[member["target"]]
    enum_values = [enum_shape["members"][member]["traits"]["smithy.api#enumValue"] for member in enum_shape["members"]]
    f.write(",".join(enum_values))
    f.write("]\n")

  # Finish writing list
  # if member_type == "list":
    # list_member_target = shapes[member["target"]]["member"]["target"]
    # list_member_type = shapes[list_member_target]
    # f.write(tabs(num_tabs+1) + "items:\n")
    # f.write(tabs(num_tabs+2) + f"type: poopy\n")

  # Write description
  description = member["traits"].get("smithy.api#documentation")
  if description is not None:
    # pass
    f.write(tabs(num_tabs+1) + f"description: {clean_docstring(description)}\n")

  # TODO Write "required" property

def write_member(member, shapes):
    terminal_types = ["string", "boolean", "integer", "timestamp", "long", "blob", "enum", "list"]
    member_type = shapes[member["target"]]["type"]
    if member_type in terminal_types:
      write_terminal_member(f, member, shapes)
    else:
      if member_type == "structure":
        f.write(tabs(4) + member["target"] + ":\n")
        f.write(tabs(5) + f"type: {convert_member_type(member_type)}\n")
        f.write(tabs(5) + "properties:\n")
        for submember_name in shapes[member["target"]]["members"]:
          submember = shapes[shapes[member["target"]]["members"][submember_name]["target"]]
          print(submember)
          if submember["type"] in terminal_types:
             write_terminal_member(f, submember, shapes, 6)
          else: raise Exception(member_type + "sub member is not terminal.")

with open("../openapi.yaml", "w") as f:
    write_openapi_header(f)
    write_paths(f, s3["shapes"])
    write_schemas(f, s3["shapes"])

Writing schema for com.amazonaws.s3#PutObjectRequest
Processing ACL
Processing Body
Processing Bucket
Processing CacheControl
Processing ContentDisposition
Processing ContentEncoding
Processing ContentLanguage
Processing ContentLength
Processing ContentMD5
Processing ContentType
Processing ChecksumAlgorithm
Processing ChecksumCRC32
Processing ChecksumCRC32C
Processing ChecksumSHA1
Processing ChecksumSHA256
Processing Expires
Processing GrantFullControl
Processing GrantRead
Processing GrantReadACP
Processing GrantWriteACP
Processing Key
Processing Metadata
Processing ServerSideEncryption
Processing StorageClass
Processing WebsiteRedirectLocation
Processing SSECustomerAlgorithm
Processing SSECustomerKey
Processing SSECustomerKeyMD5
Processing SSEKMSKeyId
Processing SSEKMSEncryptionContext
Processing BucketKeyEnabled
Processing RequestPayer
Processing Tagging
Processing ObjectLockMode
Processing ObjectLockRetainUntilDate
Processing ObjectLockLegalHoldStatus
Processing ExpectedBucketOwner
